## Getting Started

In [1]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.geoenrichment import enrich
from arcgis.features import FeatureSet

In [2]:
gis = GIS(api_key="YOU-API-KEY")

## StudyAreas

In [3]:
df = pd.read_csv("data/cvs_los_angeles.csv")
df

,Business Name,Address,City,State,ZIP
0,CVS,11051 Victory Blvd,North Hollywood,CA,91606
1,CVS,864 Swarthmore Ave,Pacific Palisades,CA,90272
2,CVS,15614 Whittwood Ln,Whittier,CA,90603
3,CVS,1025 E Adams Blvd,Los Angeles,CA,90011
4,CVS,4235 Pacific Coast Hwy,Torrance,CA,90505
...,...,...,...,...,...
489,CVS,25601 Jeronimo Rd,Mission Viejo,CA,92691
490,CVS,4040 E Chapman Ave,Orange,CA,92869
491,CVS,855 E Birch St,Brea,CA,92821
492,CVS,8601 S Sepulveda Blvd,Los Angeles,CA,90045


## Enriching Areas

In [4]:
cvs_areas = [{"address":x, "areaType": "DriveTimeBuffer","bufferRadii":[5], "bufferUnits": "Minutes"} for x in df.to_dict('records')]
cvs_areas[0]

{'address': {'Business Name': 'CVS',
  'Address': '11051 Victory Blvd',
  'City': 'North Hollywood',
  'State': 'CA',
  'ZIP': 91606},
 'areaType': 'DriveTimeBuffer',
 'bufferRadii': [5],
 'bufferUnits': 'Minutes'}

In [5]:
cvs_dt_5min = enrich(cvs_areas, analysis_variables=["AtRisk.SENIORS_CY"])
cvs_dt_5min.head(n=2)

,ID,OBJECTID,sourceCountry,X,Y,areaType,bufferUnits,bufferUnitsAlias,bufferRadii,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,SENIORS_CY,SHAPE
0,0,1,US,-118.372072,34.186855,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,12571,"{""rings"": [[[-118.36911583857336, 34.220012828..."
1,1,2,US,-118.525647,34.046369,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,3197,"{""rings"": [[[-118.54248989609434, 34.060338017..."


In [6]:
cvs_map = gis.map('Ontario, CA, USA', zoomlevel=10)
cvs_map.basemap = 'dark-gray'

cvs_map.add_layer(FeatureSet.from_dataframe(cvs_dt_5min), {'renderer':'ClassedColorRenderer','field_name':'SENIORS_CY', 'opacity':0.5})
cvs_map

MapView(layout=Layout(height='400px', width='100%'))

## Data Analysis

In [7]:
cvs_dt_5min = cvs_dt_5min.sort_values(by=['SENIORS_CY'], ascending=False)


best_csv = cvs_dt_5min[:10]
best_csv

,ID,OBJECTID,sourceCountry,X,Y,areaType,bufferUnits,bufferUnitsAlias,bufferRadii,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,SENIORS_CY,SHAPE
57,57,58,US,-118.287007,34.063138,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,24550,"{""rings"": [[[-118.28692036736885, 34.090431440..."
13,13,14,US,-118.308184,34.061863,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,22894,"{""rings"": [[[-118.30937814616557, 34.090206862..."
33,33,34,US,-118.309252,34.042647,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,20039,"{""rings"": [[[-118.30915356824286, 34.068871973..."
81,81,82,US,-118.240726,34.145775,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,20013,"{""rings"": [[[-118.22920387357054, 34.177118469..."
38,38,39,US,-118.109698,34.079326,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,19751,"{""rings"": [[[-118.10276657584576, 34.104130685..."
84,84,85,US,-118.258259,34.147669,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,19690,"{""rings"": [[[-118.25682694189477, 34.171504024..."
39,39,40,US,-118.316831,33.886669,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,18795,"{""rings"": [[[-118.30915356824286, 33.924243874..."
52,52,53,US,-118.258638,34.143076,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,17716,"{""rings"": [[[-118.25705151981748, 34.164092956..."
65,65,66,US,-118.090963,34.103599,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,17626,"{""rings"": [[[-118.09468177511961, 34.142308910..."
33,33,34,US,-117.989538,33.716020,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,17537,"{""rings"": [[[-117.98957936837462, 33.753115594..."


In [8]:
best_cvs_map = gis.map('Ontario, CA, USA', zoomlevel=10)
best_cvs_map.basemap = 'dark-gray'

best_cvs_map.add_layer(FeatureSet.from_dataframe(best_csv.copy()), {'renderer':'ClassedColorRenderer','field_name':'SENIORS_CY', 'opacity':0.5})
best_cvs_map

MapView(layout=Layout(height='400px', width='100%'))